<a href="https://colab.research.google.com/github/AlexHappyCode/wine_quality_project_team_4/blob/main/Team_4_wine_quality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wine Quality
### Team 4

https://archive.ics.uci.edu/ml/datasets/Wine+Quality

# New Section